In [47]:
from sklearn.preprocessing import PolynomialFeatures, FunctionTransformer
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.linear_model import Lasso, Ridge, ElasticNet, LinearRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import math
import pandas as pd
import numpy as np
import streamlit as st
from joblib import dump 
import joblib




df = pd.read_csv('dataset2.csv')

print(df.head(), df.shape)
# separation des features et de la variable cible
X = df.drop('charges', axis=1)
y = df[['charges']]
print(f'''verif des dimensions X et Y
      X (dataset sans la variable cible): {X.shape}
      Y (la variable cible) : {y.shape}''')


# division du dataset en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.85, random_state=42, stratify=X['smoker'])
print(f''' verif du split 80 20 
80% du dataset : X train -> {X_train.shape}, Y train -> {y_train.shape}
# 20% du dataset : X test -> {X_test.shape}, Y test -> {y_test.shape}''')


   age     sex     bmi  children smoker     region      charges
0   19  female  27.900         0    yes  southwest  16884.92400
1   18    male  33.770         1     no  southeast   1725.55230
2   28    male  33.000         3     no  southeast   4449.46200
3   33    male  22.705         0     no  northwest  21984.47061
4   32    male  28.880         0     no  northwest   3866.85520 (1337, 7)
verif des dimensions X et Y
      X (dataset sans la variable cible): (1337, 6)
      Y (la variable cible) : (1337, 1)
 verif du split 80 20 
80% du dataset : X train -> (1136, 6), Y train -> (1136, 1)
# 20% du dataset : X test -> (201, 6), Y test -> (201, 1)


In [48]:
#Elasticnet
# préprocesseur pour les variables numériques
def log_transform(x):
    return np.log(x + 1)

log_transformer = FunctionTransformer(log_transform)

preprocessor_num = Pipeline(steps=[
    ('log', FunctionTransformer(log_transform)),
    ('scaler', StandardScaler()) 
])

preprocessor_cat = Pipeline(steps=[
    ('encoder', OneHotEncoder())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', preprocessor_num, ['age', 'bmi', 'children']),
        ('cat', preprocessor_cat, ['region', 'sex', 'smoker'])
    ]
)

pipeline_lasso = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('regression', Lasso())
])

#test de différents hyperparam d'alpha pour le lasso
param_grid_lasso = {
    'regression__alpha': np.arange(7,10,0.1),
    'regression__precompute': [True],
    'regression__max_iter': [5000],
    
}

### grid search
grid_lasso = GridSearchCV(pipeline_lasso, param_grid_lasso, cv=5)


# eviter data leakage -> entraîner le pipeline sur les données d'entraînement 
grid_lasso.fit(X_train, y_train)
#puis predire y sur l'ensemble de test avec le meme pipeline
y_pred = grid_lasso.predict(X_test)
# print(y_pred)





In [49]:
#test des les differents metriques sur modele elasticnet
#comparaison du y prédit avec le y de test
mse = mean_squared_error(y_test, y_pred)

print(f'Mean Squared Error du modèle elasticnet: {mse}')

r2 = r2_score(y_test, y_pred)

print(f'Coefficient de determination R² du modèle elasticnet: {r2}')

rmse = math.sqrt(mse)
print(f'Root Mean Squared Error (RMSE) du modèle elasticnet: {rmse} \n rappel moyenne charge : 13279')

Mean Squared Error du modèle elasticnet: 13890862.143149134
Coefficient de determination R² du modèle elasticnet: 0.9041452813914066
Root Mean Squared Error (RMSE) du modèle elasticnet: 3727.044692936903 
 rappel moyenne charge : 13279


In [57]:
best_model = grid_lasso.best_estimator_

# Extraire le modèle Lasso
lasso_model = best_model.named_steps['regression']

# Obtenir les coefficients
coefficients = lasso_model.coef_
coefficients

# Étape 2 : Gérer les transformations
# Pour PolynomialFeatures
poly_features = best_model.named_steps['poly'].get_feature_names(input_features=X_train.columns)
# Pour OneHotEncoder
cat_features = best_model.named_steps['preprocessor'].named_transformers_['cat']['encoder'].get_feature_names(input_features=['region', 'sex', 'smoker'])
# Combinez les noms de toutes les features
all_features = np.concatenate([poly_features, cat_features])

# Étape 3 : Associer les coefficients aux features
feature_weights = dict(zip(all_features, coefficients))

# Étape 4 : Afficher les poids
sorted_weights = sorted(feature_weights.items(), key=lambda x: abs(x[1]), reverse=True)
for feature, weight in sorted_weights:
    print(f"{feature}: {weight}")

AttributeError: 'PolynomialFeatures' object has no attribute 'get_feature_names'

In [51]:
#best_model.coeff()

In [52]:
new_data = pd.DataFrame({
    'age': [35, 40, 30, 60,60],  
    'sex': ['female', 'male', 'male', 'female','female'],  
    'bmi': [22.5, 30.0, 25.0, 47,50],
    'children': [1, 2, 0, 5,5],
    'smoker': ['yes', 'no', 'no', 'yes','yes'],  
    'region': ['southwest', 'northeast', 'southeast', 'southeast', 'southeast'],
})

# faire des prédictions avec le modèle optimisé
new_y_ped = best_model.predict(new_data)

# ajouter les prédictions à new_data
new_data['predicted_charges'] = new_y_ped

print(new_data)

   age     sex   bmi  children smoker     region  predicted_charges
0   35  female  22.5         1    yes  southwest       17866.389731
1   40    male  30.0         2     no  northeast        9389.059658
2   30    male  25.0         0     no  southeast        4587.589497
3   60  female  47.0         5    yes  southeast       58931.050406
4   60  female  50.0         5    yes  southeast       61656.635909


In [53]:
import pickle
with open('best_model.pkl', 'wb') as fichier:
    pickle.dump(best_model, fichier)